# Part I - WeatherPy

In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world of varying distance from the equator. To accomplish this, you'll be utilizing a simple Python library, the OpenWeatherMap API, and a little common sense to create a representative model of weather across world cities.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

627

## Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).

In [3]:
# Make a request for each of the cities and capture needed data for upcoming dataframe

base_url = 'http://api.openweathermap.org/data/2.5/weather?'
units = "metric"
query_url = f'{base_url}appid={weather_api_key}&units={units}&q='

cities_found = []
response = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []
x = 0

print('Beginning Data Retrieval')
print('-----------------------------')

for city in cities:
    query = f'{query_url}{city}'
    city_response = requests.get(query).json()
    x = x + 1
    print(f'Processing Record {x} | {city}')
    
    try:
        response.append(city_response['name'])
        cities_found.append(city)
        lat.append(city_response['coord']['lat'])
        lng.append(city_response['coord']['lon'])
        max_temp.append(city_response['main']['temp_max'])
        humidity.append(city_response['main']['humidity'])
        cloudiness.append(city_response['clouds']['all'])
        wind_speed.append(city_response['wind']['speed'])
        country.append(city_response['sys']['country'])
        date.append(city_response['dt'])
        
    except:
        print('City not found. Skipping...')
        
print('-----------------------------\nData Retrieval Complete\n-----------------------------')

Beginning Data Retrieval
-----------------------------
Processing Record 1 | wabag
Processing Record 2 | busselton
Processing Record 3 | albany
Processing Record 4 | port alfred
Processing Record 5 | tecoanapa
Processing Record 6 | cabo san lucas
Processing Record 7 | alappuzha
Processing Record 8 | ushuaia
Processing Record 9 | bluff
Processing Record 10 | bethel
Processing Record 11 | tapaua
City not found. Skipping...
Processing Record 12 | vaini
Processing Record 13 | mataura
Processing Record 14 | oussouye
Processing Record 15 | tuktoyaktuk
Processing Record 16 | harigue
City not found. Skipping...
Processing Record 17 | faya
Processing Record 18 | esperance
Processing Record 19 | kichmengskiy gorodok
Processing Record 20 | cape town
Processing Record 21 | puerto ayora
Processing Record 22 | thompson
Processing Record 23 | new norfolk
Processing Record 24 | akureyri
Processing Record 25 | airai
Processing Record 26 | ahipara
Processing Record 27 | arawa
Processing Record 28 | san 

# Convert Raw Data to DataFrame

* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# create a data frame from weather data
weather_dict = {
    'City': cities_found,
    'Lat': lat,
    'Lng': lng,
    'Max Temp': max_temp,
    'Humidity': humidity,
    'Cloudiness': cloudiness,
    'Wind Speed': wind_speed,
    'Country': country,
    'Date': date
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,wabag,-5.48,143.70,13.85,94,100,0.13,PG,1592356252
1,busselton,-33.65,115.33,17.22,56,46,13.40,AU,1592356252
2,albany,42.60,-73.97,22.78,51,0,0.80,US,1592356252
3,port alfred,-33.59,26.89,13.33,82,0,2.24,ZA,1592356252
4,tecoanapa,16.52,-98.75,28.88,72,0,2.77,MX,1592356253
...,...,...,...,...,...,...,...,...,...
559,imeni poliny osipenko,52.42,136.49,11.08,78,100,3.70,RU,1592356356
560,pinhao,-25.70,-51.66,13.16,95,3,2.32,BR,1592356263
561,dicabisagan,17.08,122.42,30.24,64,100,3.08,PH,1592356356
562,upata,8.01,-62.40,26.11,81,100,2.24,VE,1592356356


In [6]:
# save weather_data data frame as csv file
weather_data.to_csv('output_data/weather_data.csv')

# Inspect the data and remove the cities where the humidity > 100%.

Skip this step if there are no cities that have humidity > 100%.